In [1]:
import pandas as pd
import numpy as np

In [2]:
# load a csv:
df = pd.read_csv('data/all-boas-april2023-march2024.csv')


In [3]:
# print columns:
print(df.columns)


Index(['ACCEPT TIME', 'ACCEPTING PARTY', 'ACCEPTED BMU', 'BOA REF', 'MAIN SP',
       'EXCLUDE BOA REASONS', 'NET VOL (MWh)', 'AVG PRICE (GBP/MWh)',
       'NOTICE TIME (MINS)', 'TIME TO TARGET (MINS)', 'TARGET DURATION (MINS)',
       'TOTAL DURATION (MINS)', 'PEAK CHANGE (MW)', 'REJECTED BOD',
       'ALTERNATIVE BMU', 'UNITS IN STACK'],
      dtype='object')


In [6]:
import pandas as pd
import json

# 1. Read all the data files
boas_df = pd.read_csv('data/all-boas-april2023-march2024.csv')
carbon_intensity_df = pd.read_csv('data/carbon_intensity.csv')

# Load BMU mapping data
with open('data/response_1733913471428.json', 'r') as f:
    bmu_data = json.load(f)

# Rest of the code remains the same
bmu_to_fuel_mapping = {}
for unit in bmu_data:
    national_grid_bmu = unit['nationalGridBmUnit']
    elexon_bmu = unit['elexonBmUnit']
    fuel_type = unit['fuelType']
    
    if fuel_type:
        # Map both National Grid and Elexon BMU codes
        bmu_to_fuel_mapping[national_grid_bmu] = fuel_type
        bmu_to_fuel_mapping[elexon_bmu] = fuel_type

# 3. Add fuel type to BOAs dataframe
boas_df['fuel_type'] = boas_df['ACCEPTED BMU'].map(bmu_to_fuel_mapping)

# 4. Standardize fuel type names to match carbon intensity data
fuel_type_mapping = {
    'WIND': 'Wind',
    'CCGT': 'Gas (Combined Cycle)',
    'OCGT': 'Gas (Open Cycle)',
    'COAL': 'Coal',
    'NUCLEAR': 'Nuclear',
    'PS': 'Pumped Storage',
    'BIOMASS': 'Biomass',
    'SOLAR': 'Solar',
    'HYDRO': 'Hydro',
    'OIL': 'Oil'
}
boas_df['fuel_type'] = boas_df['fuel_type'].map(fuel_type_mapping).fillna('Other')

# 5. Merge with carbon intensity data and calculate emissions
result_df = boas_df.merge(carbon_intensity_df, on='fuel_type', how='left')
result_df['carbon_emissions_g'] = result_df['NET VOL (MWh)'].fillna(0) * result_df['carbon_intensity'] * 1000
result_df['carbon_emissions_tonnes'] = result_df['carbon_emissions_g'] / 1_000_000

# 6. Print analysis
print("\nTotal carbon emissions (tonnes):", f"{result_df['carbon_emissions_tonnes'].sum():,.2f}")
print("\nEmissions by fuel type:")
print(result_df.groupby('fuel_type')['carbon_emissions_tonnes'].sum().sort_values(ascending=False))

# 7. Check for unmapped BMUs
unmapped_bmus = boas_df[boas_df['fuel_type'] == 'Other']['ACCEPTED BMU'].unique()
print(f"\nNumber of unmapped BMUs: {len(unmapped_bmus)}")
print("\nSample of unmapped BMUs (first 10):")
print(unmapped_bmus[:10])

# Optional: Save results to CSV
result_df.to_csv('data/carbon_emissions_results.csv', index=False)


Total carbon emissions (tonnes): 1,925,506.71

Emissions by fuel type:
fuel_type
Gas (Combined Cycle)    1.813812e+06
Coal                    1.361713e+05
Biomass                 6.792708e+03
Pumped Storage          0.000000e+00
Wind                    0.000000e+00
Gas (Open Cycle)       -2.177773e+03
Other                  -2.909107e+04
Name: carbon_emissions_tonnes, dtype: float64

Number of unmapped BMUs: 212

Sample of unmapped BMUs (first 10):
['CAS-GAR01' 'POTES-1' 'FASN3' 'FASN2' 'FFSE01' 'BBPG01' 'AG-HEL0CP'
 'NURSB-1' 'AG-HEL0DN' 'FARNB-1']
